##**SSAFY GPT-2 project**
본 실습은 Sub PJT3에 앞서 GPT-2의 동작 방법의 이해를 돕기위한 실습입니다.
GPT-2는 Open AI에서 2018년에 공개한 거대 자연어 처리 모델입니다. Transformer 디코더 구조를 사용하여 전체 모델을 구성하였고, 인터넷에서 얻은 대용량 데이터셋에 대해서 학습이 되어 다양한 자연어 처리에서 좋은 성능을 보이고 있습니다. Sub PJT3에서 GPT-2를 사용하여 이미지 캡셔닝을 구현할 예정이기 때문에 본 실습을 통해 GPT-2를 확실히 이해해야 합니다. 실습은 2가지 내용으로 구성되어 있습니다.
- GPT-2 모델 및 tokenizer 실습
- Special token 추가하는 실습

### **Transformer 패키지 설치 및 필요한 패키지 import**

In [ ]:
!pip install transformers

In [ ]:
from transformers import *
from torch import nn
from tqdm import tqdm

import torch

### **GPT-2 불러오기**

In [ ]:
gpt_name = 'gpt2'

In [ ]:
# transformers 패키지로부터 GPT-2 모델을 불러옵니다
config = GPT2Config.from_pretrained(gpt_name)
tokenizer = GPT2Tokenizer.from_pretrained(gpt_name)
model = GPT2Model.from_pretrained(gpt_name)

In [ ]:
# GPT-2를 구성하는 configuration을 확인합니다.
# n_ctx : 최대 input 길이, max embedding position
# n_embd : hidden size
config

In [ ]:
# Tokenizer는 입력문장을 일정한 단위로 분할하는 역할을 합니다. 
# GPT-2는 transformer decoder로만 구성되어 있어 masked multi-head attention 수행합니다.
# 즉, 왼쪽부터 하나씩 token을 생성하는 방식이기 때문에 padding 토큰은 따로 없습니다.

tokenizer

In [ ]:
# GPT-2 모델의 구조를 출력해봅니다.
# 각 GPT2Block은 transformer 구조로 되어 있는 것을 확인할 수 있습니다.
model

### **Tokenizer 사용**

- 앞서 설명한대로, tokenizer는 입력문장을 일정한 단위로 분할하는 역할을 합니다. 

In [ ]:
# 아래 문장의 단어를 각각의 token으로 tokenize한 결과입니다.
# tokenize한 뒤 vocab의 id형태로 출력되는 것을 확인할 수 있습니다.

sentence = "I want to go home."
output = tokenizer(sentence)
output

In [ ]:
# 위에처럼 index 형태가 아닌, 자연어 형태로 tokenize 결과를 출력합니다.
# GPT-2 tokenizer는 단어 사이에 띄어쓰기도 하나의 char로 인식하도록 학습되어 있습니다.
# Ġ표기 이외에 위의 cell의 sentence와 동일한 결과를 확인할 수 있습니다.
# Ġ는 GPT-2 일종의 special한 표기입니다. 이는 띄어쓰기 표시를 의미합니다.

tokenized = tokenizer.tokenize(sentence)
tokenized

In [ ]:
# Tokenizer에 저장되어 있는 vocab을 불러와서 출력해봅니다.
# 표현 가능한 Vocab은 총 50257개 입니다.
# 특수 문자들도 포함되어 있는 것을 확인할 수 있습니다.
vocab = tokenizer.get_vocab()

print(vocab)
print(len(vocab))

In [ ]:
# vocab의 가장 마지막에 있는 token은 아래와 같고, id가 마지막을 가리키는 것을 확인할 수 있습니다.
# 사용자가 start, end token 필요할때 해당 token을 사용할 수 있습니다.

vocab['<|endoftext|>']

In [ ]:
# 이제 다시, 위에서 작성한 "I want to go home"이라는 문장을 tokenize를 거친 결과는
# vocab 내의 token 순서로 표시된 것을 확인할 수 있습니다.

token_ids = [vocab[token] for token in tokenized]
print(token_ids)

In [ ]:
# tokenizer 내의 _convert_token_to_id() 함수로 token id를 추출할 수 있습니다.

token_ids = [tokenizer._convert_token_to_id(token) for token in tokenized]
print(token_ids)

In [ ]:
# tokenizer 내의 convert_token_to_ids() 함수를 사용하면, 바로 문장의 모든 token을 담음
# tokenized를 입력하여 token id를 추출할 수 있습니다.

token_ids = tokenizer.convert_tokens_to_ids(tokenized)
print(token_ids)

In [ ]:
# tokenizer 내의 encode() 함수를 사용하면 입력 문장(string)으로 부터 
# 바로 token id를 추출할 수 있습니다.

token_ids = tokenizer.encode(sentence)
print(token_ids)

In [ ]:
# tokenizer 내의 convert_tokens_to_string() 함수를 사용하면 
# tokenized 결과로 부터 자연어 형태의 원본 문장을 얻을 수 있습니다.

sentence = tokenizer.convert_tokens_to_string(tokenized)
print(sentence)

In [ ]:
# convert_ids_to_tokens()와 convert_tokens_to_string의 차이를 확인해 봅니다.

tokens = tokenizer.convert_ids_to_tokens(token_ids)
print(tokens)
sentence = tokenizer.convert_tokens_to_string(tokens)
print(sentence)

### **데이터 전처리**

데이터를 전처리를 전처리 하는 방법에 대해서 실습해 봅니다. GPT-2에 batch 단위로 데이터를 넘겨줄 때 batch내의 모든 문장의 길이는 동일해야 합니다. 실제로 문장의 길이가 동일하지 않는 경우가 있기 때문에, 가장 긴 문장을 기준으로 나머지 문장들에 임의의 token을 넣어주어 길이가 동일하게 만듭니다.

### Req. 3-1	데이터 전처리하기: batch 형태로 만들기

In [ ]:
data = [
  "I want to go home.",
  "My dog's name is Max.",
  "Natural Language Processing is my favorite research field.",
  "Welcome. How can I help you?",
  "Shoot for the moon. Even if you miss, you'll land among the stars."
]

In [ ]:
# 전체 길이 맞추기 위해 max_len 구하고
# 각각의 문장을 encode하고 max_len 구한다

# GPT-2는 별도 pad token 없고, 어차피 무시할 수 있기 때문에
# 아무 token이나 넣어도 된다
# 여기서는 0번으로 padding

max_len = 0
batch = []

################################################################################
# TODO: 데이터를 batch 형태로 만들기.                                                #
################################################################################
# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
# 1) data를 token_id 형태로 batch 리스트에 append 합니다.
# 2) batch 내에서 가장 긴 문장의 max_len를 구합니다.
# 3) batch 내의 모든 데이터 길이를 동일하게 만들기 위해서 max len에 도달할 때 까지
# 임의의 token으로 채워줍니다. 여기서는 0 token으로 채워줍니다.

# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
################################################################################
#                                 END OF YOUR CODE                             #
################################################################################

In [ ]:
batch = torch.LongTensor(batch)

print(batch)
print(batch.shape)

### Req. 3-2	데이터 전처리하기: batch 내 불필요한 부분 masking

In [ ]:
################################################################################
# TODO: batch 내 불필요한 부분 masking.                                            #
################################################################################
# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
# 1) 길이를 맞춰주기 위해서 임의로 token을 생성한 부분은 사실 불필요한 부분입니다.
# 2) 추후에 해당 부분을 무시할 수 있도록 masking할수 있는 변수를 만듭니다.
# 3) 문장에 해당하는 부분은 1, 무시할 부분은 0으로 채워진 tensor를 생성하고
# 이는 batch와 tensor shape이 동일해야 합니다.

# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
################################################################################
#                                 END OF YOUR CODE                             #
################################################################################

print(batch_mask)
print(batch.shape)

### **GPT-2 사용 및 응용**

In [ ]:
# 앞서 생성한 batch와 batch_mask를 모델에 넣어주면 아래와 같은 결과를 얻을 수 있습니다.

outputs = model(input_ids=batch, attention_mask=batch_mask)
outputs

In [ ]:
# 앞선 결과는 GPT-2의 모든 layer의 attention 거치고 나온 최종 결과입니다.
# Batch 내에서 하나를 선정하여 tensor shape을 확인합니다.

# B: batch size, L: max length, d_h: hidden size
last_hidden_states = outputs[0]  # (B, L, d_h)

print(last_hidden_states.shape)

- GPT-2는 next word prediction이라고 하는, 다음 단어를 예측하는 방식으로 기학습된 자연어 처리 모델입니다.
- 실제로 현재 token 다음에 나오는 token이 무엇인지 예측하고 싶다면 linear layer를 추가하여 확인할 수 있습니다.
- 아래와 같이 hidden_size를 입력받아, vocab size 크기의 dimension을 출력하는 fully-connected layer를 사용하여 다음 단어를 예측할 수 있습니다.


In [ ]:
lm_linear = nn.Linear(config.hidden_size, config.vocab_size)

In [ ]:
# vocab_size의 크기로 출력하는 이유는, 다음으로 등장할 단어가 무엇인지
# 모든 단어에 대해서 확률을 구하고 싶기 때문입니다.
# 실제로는 vocab size로 출력된 값 중 가장 큰 값을 뽑아서 예측할 수 있습니다.
# V: vocab size

lm_output = lm_linear(last_hidden_states)  # (B, L, V)

print(lm_output)
print(lm_output.shape)

GPT-2 는 위의 예시와 같이 다양한 head를 제공해줍니다. (https://huggingface.co/transformers/model_doc/gpt2.html)  
위의 Language Modeling을 동일하게 수행할 수 있는 모델은 아래와 같습니다.

In [ ]:
lm_model = GPT2LMHeadModel.from_pretrained(gpt_name)

In [ ]:
# 가장 마지막 lm_head 레이어에 다음 단어를 예측하는 레이어까지 포함되어 있는 것을 확인할 수 있습니다.
lm_model

GPT2LMHeadModel은 `input_ids`와 `labels`를 함께 줄 경우 학습 중이라고 인식하고 자동으로 cross entropy loss까지 계산합니다.  
`labels`가 주어지지 않을 경우엔 기존과 동일하게 결과만 출력합니다.

In [ ]:
# input_ids, labels에 같은 값 batch를 넣습니다.
# batch를 한 번 자동으로 shift해서 모델이 next word prediction을 얼마나 잘 하고있는지 
# cross-entropy loss까지 계산해봅니다.

outputs = lm_model(input_ids=batch, attention_mask=batch_mask, labels=batch)
outputs

In [ ]:
# 위 출력의 첫번째 index에는 loss 값을 담고 있습니다.
# 학습을 원할 경우 아래의 갚은 backpropagation 해서 사용합니다.

loss = outputs[0]

In [ ]:
# 위 출력의 두번째 index에는 logit 값을 담고 있습니다.
# logit은 모델으 마지막 레이어를 거쳐서 torch.Size([5, 18, 50257]) shape을 갖습니다.
# vocab size: 50257

logits = outputs[1]

print(logits)
print(logits.shape)

### **Special token 추가하기**

경우에 따라선 별도의 special token을 추가하고 싶을 수 있습니다. 
- 예를 들어 챗봇을 만들때 각 화자 정보를 나타내는 special token 필요
- Begining of sentence (BOS), End of sentence (EOS)를 하나로 퉁치지 않고 구분하고 싶은 경우

In [ ]:
print(vocab)
print(len(vocab))

In [ ]:
# dictionary - tokenizer 한테 추가할 special token 정보 제공하는 형식

special_tokens = {
    'bos_token': '[BOS]',
    'eos_token': '[EOS]',
    'pad_token': '[PAD]',
    'additional_special_tokens': ['[SP1]', '[SP2]']
}

In [ ]:
# tokenizer 내 add_special_tokens 함수로 새로우 token을 추가합니다.
# 자동으로 tokenizer와 그 안의 vocab에 special token 등록되어 새롭게 등장한 token들의 수가 반환됩니다.

num_new_tokens = tokenizer.add_special_tokens(special_tokens)
print(num_new_tokens)

In [ ]:
# 50257 -> 50262, 5개 증가

vocab = tokenizer.get_vocab()
print(vocab)
print(len(vocab))

In [ ]:
tokenizer

현재 vocab과 tokenizer만 바뀐 상황이고, 모델은 이전의 vocab size와 embedding row를 가집니다. 따라서 모델의 embedding size도 바뀐 vocab과 tokenizer에 맞게 수정해주어야 합니다.


In [ ]:
# resize_token_embeddings
# 자체적으로 model 내에서 embedding layer 조절해준다

model.resize_token_embeddings(len(vocab))

In [ ]:
# Embedding의 사이즈가 바뀐것을 확인할 수 있습니다.
# 그러나 아직 바뀐 다음에 학습이 안되었기 때문에 추후 finetuning 단계가 필요합니다.
model